In [7]:
import functools
import pickle

import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display

RUNDIRS = '../logs/rundirs'

In [ ]:
@functools.cache
def get_key2df(runname):
    rundir = f'{RUNDIRS}/{runname}'
    with open(f'{rundir}/key2df.pickle', 'rb') as file:
        return pickle.load(file)

In [6]:
def plot_df(*, runname, df, col_strategy, strategy, col_i, col_j, col_data):
    df = df[df.index.get_level_values(col_strategy) == strategy]
    heatmap_data = df[[col_data]].reset_index().pivot(
        index=col_i, columns=col_j, values=col_data
    )

    # Create an interactive heatmap
    fig = px.imshow(
        heatmap_data,
        labels={"x": col_j, "y": col_i, "color": col_data},
        title=f"{runname}: {strategy}: {col_data}",
    )
    
    fig.update_traces(
        hovertemplate=(
            col_i + ": %{y}<br>" + col_j + ": %{x}<br>" + col_data + ": %{z}<extra></extra>"
        )
    )
    
    return heatmap_data, fig


def plot_df_all(runname, are_bridges, label_are_bridges):
    key2df = get_key2df(runname)
    df = key2df[are_bridges, True]
    display(df)
    display(df.columns)
    df = df.drop(columns=['i_map', 'Positions variant', 'Coordination strategy']).reset_index()
    df['Coordination strategy'] = df['Coordination strategy'].apply(lambda s: 'baseline' if s == 'no' else s)
    df = df.sort_values(by=['i_map', 'Positions variant', 'Coordination strategy'])
    df = df.set_index(['i_map', 'Positions variant', 'Coordination strategy'])
    
    # df = df[~df['passhum']]
    
    col_strategy = 'Coordination strategy'
    col_i = 'i_map'
    col_j = 'Positions variant'
    col_data ='No. of completed missions'
    
    strategies = df.index.get_level_values(col_strategy).unique()
    
    # Create subplots
    fig = make_subplots(
        rows=1,
        cols=len(strategies),
        subplot_titles=[f"{strategy}" for strategy in strategies],
        #shared_yaxes=True
    )
    
    for idx, strategy in enumerate(strategies):
        heatmap_data, heatmap_fig = plot_df(
            runname=runname, df=df, 
            col_strategy=col_strategy, strategy=strategy, 
            col_i=col_i, col_j=col_j, col_data=col_data,
        )
          
        # Add heatmap to the subplot
        for trace in heatmap_fig.data:
            trace.update(coloraxis="coloraxis")  # Link each subplot to the shared color axis
            fig.add_trace(trace, row=1, col=idx + 1)
        
        # Apply axis settings to each subplot
        xaxis_key = f"xaxis{idx + 1}" if idx > 0 else "xaxis"
        yaxis_key = f"yaxis{idx + 1}" if idx > 0 else "yaxis"
        
        fig.update_layout(**{
            xaxis_key: dict(
                title=col_j,
                tickmode="array",
                tickvals=list(heatmap_data.columns),
                ticktext=heatmap_data.columns.tolist(),
            ),
            yaxis_key: dict(
                title=col_i,
                tickmode="array",
                tickvals=list(heatmap_data.index),
                ticktext=heatmap_data.index.tolist(),
                autorange="reversed",  # Reverse the y-axis for top-to-bottom ticks
            )
        })
    
    # Update layout with shared color scale
    fig.update_layout(
        title=f"{col_data} ({runname}, {col_strategy}, {label_are_bridges})",
        coloraxis=dict(
            colorscale="Greens",  # Apply "Greens" color scale to the shared color axis
            colorbar=dict(
                title=col_data,
                titleside="right",  # Position the colorbar title vertically to the left
                titlefont=dict(size=12)  # Optional: Adjust font size for better appearance
            )
        )
    )
    
    # Display the figure
    fig.show()



# Call the function
plot_df_all('20241230_173555', are_bridges=False, label_are_bridges='maps with low connectivity')
plot_df_all('20241230_173555', are_bridges=True, label_are_bridges='maps with high connectivity')

"""
- No. of completed missions
- Average mission length
- Average CS density score

Hypotheses:f
- less Average mission length -> more No. of completed
- less Average CS density score -> more No. of completed missions (& less collisions, etc.) 
"""

filename  \
i_map Positions variant Coordination strategy                                                      
1     1                 baseline               map-generator/generated-maps/2024-11-28_13:19:...   
                        change of priorities   map-generator/generated-maps/2024-11-28_13:19:...   
                        stops                  map-generator/generated-maps/2024-11-28_13:19:...   
      2                 baseline               map-generator/generated-maps/2024-11-28_13:19:...   
                        change of priorities   map-generator/generated-maps/2024-11-28_13:19:...   
...                                                                                          ...   
10    9                 change of priorities   map-generator/generated-maps/2024-11-28_13:19:...   
                        stops                  map-generator/generated-maps/2024-11-28_13:19:...   
      10                baseline               map-generator/generated-maps/2024-11-28_13:19:...   
                        change of priorities   map-generator/generated-maps/2024-11-28_13:19:...   
                        stops                  map-generator/generated-maps/2024-11-28_13:19:...   

                                                                                         dir_map  \
i_map Positions variant Coordination strategy                                                      
1     1                 baseline               2024-11-28_13:19:18_without_bridges2024-11-28_...   
                        change of priorities   2024-11-28_13:19:18_without_bridges2024-11-28_...   
                        stops                  2024-11-28_13:19:18_without_bridges2024-11-28_...   
      2                 baseline               2024-11-28_13:19:18_without_bridges2024-11-28_...   
                        change of priorities   2024-11-28_13:19:18_without_bridges2024-11-28_...   
...                                                                                          ...   
10    9                 change of priorities   2024-11-28_13:19:18_without_bridges2024-11-28_...   
                        stops                  2024-11-28_13:19:18_without_bridges2024-11-28_...   
      10                baseline               2024-11-28_13:19:18_without_bridges2024-11-28_...   
                        change of priorities   2024-11-28_13:19:18_without_bridges2024-11-28_...   
                        stops                  2024-11-28_13:19:18_without_bridges2024-11-28_...   

                                                                               basename_scenario  \
i_map Positions variant Coordination strategy                                                      
1     1                 baseline               scenario1-1scenario1-1scenario1-1scenario1-1sc...   
                        change of priorities   scenario1-1scenario1-1scenario1-1scenario1-1sc...   
                        stops                  scenario1-1scenario1-1scenario1-1scenario1-1sc...   
      2                 baseline               scenario1-2scenario1-2scenario1-2scenario1-2sc...   
                        change of priorities   scenario1-2scenario1-2scenario1-2scenario1-2sc...   
...                                                                                          ...   
10    9                 change of priorities   scenario10-9scenario10-9scenario10-9scenario10...   
                        stops                  scenario10-9scenario10-9scenario10-9scenario10...   
      10                baseline               scenario10-10scenario10-10scenario10-10scenari...   
                        change of priorities   scenario10-10scenario10-10scenario10-10scenari...   
                        stops                  scenario10-10scenario10-10scenario10-10scenari...   

                                               i_map  Positions variant  \
i_map Positions variant Coordination strategy                             
1     1                 baseline   

Index(['filename', 'dir_map', 'basename_scenario', 'i_map',
       'Positions variant', 'passhum', 'slowness', 'Coordination strategy',
       'filename_screenshot', 'are_bridges', 'configuration',
       'isCanPassFirstHum', 'isCanPassFirstAut', 'reroutingsAtParked',
       'reroutingsAtSlow', 'Date', 'Scenario ID', 'Vehicle ID', 'Vehicle type',
       'Cycle distance (m)', 'No. of completed missions',
       'Total distance traveled (m)', 'No. of stops', 'No. of forcing events',
       'No. of violations', 'No. of critical sections', 'No. of near-misses',
       'No. of collisions', 'Total waiting time (s)',
       'Maximum waiting time (s)', 'Total time (s)',
       'Maximum acceleration (m/s^2)', 'Maximum speed (m/s)',
       'Average speed (m/s)', 'Scenario', 'Current datetime',
       'Time passed (real)', 'Time passed (sim.)', 'isCanPassFirstActive',
       'isRacingThroughCrossroadAllowed', 'probabilitySlowingDownForHuman',
       'Human V0', 'Collision events', 'Vehicle size (

filename  \
i_map Positions variant Coordination strategy                                                      
1     1                 baseline               map-generator/generated-maps/2024-11-28_13:17:...   
                        change of priorities   map-generator/generated-maps/2024-11-28_13:17:...   
                        stops                  map-generator/generated-maps/2024-11-28_13:17:...   
      2                 baseline               map-generator/generated-maps/2024-11-28_13:17:...   
                        change of priorities   map-generator/generated-maps/2024-11-28_13:17:...   
...                                                                                          ...   
10    9                 change of priorities   map-generator/generated-maps/2024-11-28_13:17:...   
                        stops                  map-generator/generated-maps/2024-11-28_13:17:...   
      10                baseline               map-generator/generated-maps/2024-11-28_13:17:...   
                        change of priorities   map-generator/generated-maps/2024-11-28_13:17:...   
                        stops                  map-generator/generated-maps/2024-11-28_13:17:...   

                                                                                         dir_map  \
i_map Positions variant Coordination strategy                                                      
1     1                 baseline               2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
                        change of priorities   2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
                        stops                  2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
      2                 baseline               2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
                        change of priorities   2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
...                                                                                          ...   
10    9                 change of priorities   2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
                        stops                  2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
      10                baseline               2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
                        change of priorities   2024-11-28_13:17:39_with_bridges2024-11-28_13:...   
                        stops                  2024-11-28_13:17:39_with_bridges2024-11-28_13:...   

                                                                               basename_scenario  \
i_map Positions variant Coordination strategy                                                      
1     1                 baseline               scenario1-1scenario1-1scenario1-1scenario1-1sc...   
                        change of priorities   scenario1-1scenario1-1scenario1-1scenario1-1sc...   
                        stops                  scenario1-1scenario1-1scenario1-1scenario1-1sc...   
      2                 baseline               scenario1-2scenario1-2scenario1-2scenario1-2sc...   
                        change of priorities   scenario1-2scenario1-2scenario1-2scenario1-2sc...   
...                                                                                          ...   
10    9                 change of priorities   scenario10-9scenario10-9scenario10-9scenario10...   
                        stops                  scenario10-9scenario10-9scenario10-9scenario10...   
      10                baseline               scenario10-10scenario10-10scenario10-10scenari...   
                        change of priorities   scenario10-10scenario10-10scenario10-10scenari...   
                        stops                  scenario10-10scenario10-10scenario10-10scenari...   

                                               i_map  Positions variant  \
i_map Positions variant Coordination strategy                             
1     1                 baseline   

Index(['filename', 'dir_map', 'basename_scenario', 'i_map',
       'Positions variant', 'passhum', 'slowness', 'Coordination strategy',
       'filename_screenshot', 'are_bridges', 'configuration',
       'isCanPassFirstHum', 'isCanPassFirstAut', 'reroutingsAtParked',
       'reroutingsAtSlow', 'Date', 'Scenario ID', 'Vehicle ID', 'Vehicle type',
       'Cycle distance (m)', 'No. of completed missions',
       'Total distance traveled (m)', 'No. of stops', 'No. of forcing events',
       'No. of violations', 'No. of critical sections', 'No. of near-misses',
       'No. of collisions', 'Total waiting time (s)',
       'Maximum waiting time (s)', 'Total time (s)',
       'Maximum acceleration (m/s^2)', 'Maximum speed (m/s)',
       'Average speed (m/s)', 'Scenario', 'Current datetime',
       'Time passed (real)', 'Time passed (sim.)', 'isCanPassFirstActive',
       'isRacingThroughCrossroadAllowed', 'probabilitySlowingDownForHuman',
       'Human V0', 'Collision events', 'Vehicle size (

'\n- No. of completed missions\n- Average mission length\n- Average CS density score\n\nHypotheses:f\n- less Average mission length -> more No. of completed\n- less Average CS density score -> more No. of completed missions (& less collisions, etc.) \n'